In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
import os

In [2]:
cols=['chiranjeevi','modi','nagarjuna','rajinikanth']
links=[]
target=[]
df=pd.DataFrame()
for col in cols:
    for i in os.listdir(col):
        links.append(os.path.join(col,i))
    target+=[col]*len(os.listdir(col))
df['links']=links
df['target']=target

In [3]:
def face_of_image(link):
    test=face_recognition.load_image_file(link)
    face_location=face_recognition.face_locations(test)
    if len(face_location)==0:
        return None
    for top,right,bottom,left in face_location:
        test=test[top:bottom,left:right]
        return test/255.0
    return None

In [4]:
def detect_face(image):
    face_model = cv2.CascadeClassifier('haar_face.xml')
    coordinates = face_model.detectMultiScale(image)
    if len(coordinates) == 0:
        return None  # If no face detected, return None
    for (x, y, w, h) in coordinates:
        face = image[y:y+h, x:x+w]
        return face/255.0
    return None  # Return None if no face is detected




def get_image(link):
    image = cv2.imread(link)
    if image is None:
        print(f"Error loading image: {link}")
        return None
    image_face = face_of_image(link)
    if image_face is not None:
        try:
            image_face = cv2.resize(image_face, (100, 100))
            return image_face
        except cv2.error as e:
            print(f"Error resizing image: {link}, {e}")
            return None
    else:
        try:
            image = cv2.resize(image, (100, 100))
            return image
        except cv2.error as e:
            print(f"Error resizing image: {link}, {e}")
            return None

df['image'] = df['links'].apply(lambda x: get_image(x))
df = df[df['image'].notnull()]  # Remove rows where image processing failed


In [5]:
df.shape

(150, 3)

In [19]:
df=df.loc[np.random.permutation(df.index)]

In [7]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPooling2D
from tensorflow.keras import Sequential
model=Sequential([
    Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'),
    Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'),
    Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(200,activation='relu'),
    Dense(4,activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
df.target=df.target.map({'chiranjeevi':0,'modi':1,'nagarjuna':2,'rajinikanth':3})

In [9]:
X = np.array(df.image.tolist())
y = df.target
X=X.astype('float32')
print(X.dtype)
print(y.dtype)

float32
int64


In [10]:
y=pd.get_dummies(y)

In [12]:
history=model.fit(X,y,epochs=20)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.2843 - loss: 34.9354
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.3503 - loss: 5.4175
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 980ms/step - accuracy: 0.3369 - loss: 2.7241
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 935ms/step - accuracy: 0.2795 - loss: 1.6549
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 929ms/step - accuracy: 0.3539 - loss: 1.4000
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 922ms/step - accuracy: 0.3131 - loss: 1.3731
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 934ms/step - accuracy: 0.3780 - loss: 1.3110
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 957ms/step - accuracy: 0.3644 - loss: 1.3101
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 925ms/step - accuracy: 0.5341 - loss: 1.2356
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 930ms/step - accuracy: 0.5329 - loss: 1.1858
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 921ms/step - accuracy: 0.5962 - loss: 1.1191
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 928ms/step - accuracy: 0.6405 - loss: 0

In [13]:
mapp={0:'chiranjeevi',1:'modi',2:'nagarjuna',3:'rajinikanth'}

In [14]:
y_test=model.predict(X)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 339ms/step


In [17]:
def get_name(link):
    image=get_image(link)
    test_image=image.reshape(-1,100,100,3)
    y_pred=model.predict(test_image)
    name=mapp[y_pred.argmax()]
    return [image,name]
image,name=get_name("rajinikanth_test_image.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


In [18]:
cv2.putText(image,name,(10,95),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255))
cv2.imshow('image',image)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
y=y.apply(lambda x: x.idxmax(),axis=1)

In [20]:
y_ped=[x.argmax() for x in y_test]

In [21]:
import joblib
joblib.dump(model,'face_recognizer.joblib')

['face_recognizer.joblib']